# Tutorial: Ammonia Health Impact Analysis

This notebook is a production-grade, ready-to-run walkthrough for **multi-standard health impact assessment** of an ammonia release using `pyELDQM`.

It covers:
- scenario configuration (source, weather, grid, threshold sets)
- Gaussian dispersion simulation
- loading health thresholds from multiple standards: AEGL, ERPG, PAC, IDLH
- zone extraction for every active threshold
- distance reporting per standard and level
- interactive Folium map with all health threshold zones displayed inline

## 1) Runtime Notes

This notebook supports two execution modes:
1. **Installed package mode** (`pip install pyeldqm`)
2. **Workspace mode** (running inside the project repository)

The import cell automatically detects and handles both.

In [1]:
from __future__ import annotations
import sys
from math import radians, cos, sin, asin, sqrt
from pathlib import Path
from datetime import datetime
import numpy as np
import folium
from shapely.geometry import mapping as geom_mapping
from IPython.display import display, Markdown, HTML

IMPORT_MODE = "package"
try:
    from pyeldqm.core.dispersion_models.gaussian_model import calculate_gaussian_dispersion
    from pyeldqm.core.meteorology.stability import get_stability_class
    from pyeldqm.core.chemical_database import ChemicalDatabase
    from pyeldqm.core.utils.features import setup_computational_grid
    from pyeldqm.core.utils.zone_extraction import extract_zones
    from pyeldqm.core.health_thresholds import (
        get_aegl_thresholds,
        get_erpg_thresholds,
        get_pac_thresholds,
        get_idlh_threshold,
    )
except ModuleNotFoundError:
    IMPORT_MODE = "workspace"
    cwd = Path.cwd().resolve()
    candidates = [cwd] + list(cwd.parents)
    pkg_dir = next((p / "pyeldqm" for p in candidates if (p / "pyeldqm").exists()), None)
    if pkg_dir is None:
        raise RuntimeError(
            "Could not resolve project root. Run from repository root or install pyeldqm via pip."
        )
    project_root = pkg_dir.parent
    if str(project_root) not in sys.path:
        sys.path.insert(0, str(project_root))

    from pyeldqm.core.dispersion_models.gaussian_model import calculate_gaussian_dispersion
    from pyeldqm.core.meteorology.stability import get_stability_class
    from pyeldqm.core.chemical_database import ChemicalDatabase
    from pyeldqm.core.utils.features import setup_computational_grid
    from pyeldqm.core.utils.zone_extraction import extract_zones
    from pyeldqm.core.health_thresholds import (
        get_aegl_thresholds,
        get_erpg_thresholds,
        get_pac_thresholds,
        get_idlh_threshold,
    )

display(Markdown(f"**Import mode:** `{IMPORT_MODE}`"))


**Import mode:** `workspace`

## 2) Scenario Configuration

Update these values to model your own scenario.

In [13]:
# Chemical
CHEMICAL_NAME = "AMMONIA"
MOLECULAR_WEIGHT = 17.03              # g/mol
# Release location
SOURCE_LAT = 31.6911
SOURCE_LON = 74.0822
TIMEZONE_OFFSET_HRS = 5.0
# Release parameters
RELEASE_TYPE = "single"               # single | multi
SOURCE_TERM_MODE = "continuous"       # continuous | instantaneous
RELEASE_RATE = 800.0                  # g/s (continuous)
TANK_HEIGHT = 3.0                     # m
DURATION_MINUTES = 30.0               # min
MASS_RELEASED_KG = 500.0              # kg (instantaneous mode)
TERRAIN_ROUGHNESS = "URBAN"           # URBAN | RURAL
RECEPTOR_HEIGHT_M = 1.5               # m
# Weather
WEATHER_MODE = "manual"
WIND_SPEED = 2.5                      # m/s
WIND_DIRECTION = 90.0                 # degrees
TEMPERATURE_C = 25.0                  # C
HUMIDITY = 65.0                       # %
CLOUD_COVER = 30.0                    # %
# Computational grid
X_MAX = 12000
Y_MAX = 4000
NX = 500
NY = 500
# Health threshold sets to include
# Supported values: 'aegl', 'erpg', 'pac', 'idlh'
SELECTED_SETS = ["aegl", "erpg"]      # add/remove sets as required

assert NX > 10 and NY > 10, "Grid resolution is too low."
assert X_MAX > 0 and Y_MAX > 0, "Grid extents must be positive."
assert WIND_SPEED > 0, "Wind speed must be > 0."
assert len(SELECTED_SETS) > 0, "At least one threshold set must be selected."
print("Scenario configured successfully.")
print(f"Threshold sets       : {', '.join(s.upper() for s in SELECTED_SETS)}")
print("All outputs will be shown inline in this notebook.")

Scenario configured successfully.
Threshold sets       : AEGL, ERPG
All outputs will be shown inline in this notebook.


## 3) Helper Functions

Distance utility for reporting maximum zone reach from the source.

In [14]:
def haversine_km(lat1, lon1, lat2, lon2):
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    return 2 * asin(sqrt(a)) * 6371.0


def max_distance_from_source_km(polygon, src_lat, src_lon):
    if polygon is None or polygon.is_empty:
        return None
    max_d = 0.0
    try:
        for lon, lat in polygon.exterior.coords:
            d = haversine_km(src_lat, src_lon, lat, lon)
            if d > max_d:
                max_d = d
    except Exception:
        return None
    return max_d if max_d > 0 else None

## 4) Chemical Properties

Fetch core records from the chemical database for transparency and traceability.

In [15]:
print("=" * 72)
print("CHEMICAL PROPERTIES")
print("=" * 72)
try:
    db = ChemicalDatabase()
    chem_data = next(
        (c for c in db.get_all_chemicals() if c.get("name") == CHEMICAL_NAME),
        None,
    )
    if chem_data:
        fields = [
            ("Name",                  "name"),
            ("Molecular Weight",      "molecular_weight"),
            ("Boiling Point (K)",     "boiling_point_K"),
            ("Density (kg/m3)",       "density_kgm3"),
            ("Vapor Pressure (kPa)",  "vapor_pressure_kPa"),
            ("IDLH (ppm)",            "idlh_ppm"),
            ("LC50 (ppm)",            "lc50_ppm"),
            ("AEGL-1 (60 min)",       "aegl1_60min"),
            ("AEGL-2 (60 min)",       "aegl2_60min"),
            ("AEGL-3 (60 min)",       "aegl3_60min"),
            ("ERPG-1",                "erpg1"),
            ("ERPG-2",                "erpg2"),
            ("ERPG-3",                "erpg3"),
        ]
        for label, key in fields:
            value = chem_data.get(key)
            if value not in (None, "", 0):
                print(f"{label:<26}: {value}")
    else:
        print(f"Name                     : {CHEMICAL_NAME}")
        print(f"Molecular Weight         : {MOLECULAR_WEIGHT} g/mol")
        print("Database record not found; using configured molecular weight.")
except Exception as ex:
    print(f"Name                     : {CHEMICAL_NAME}")
    print(f"Molecular Weight         : {MOLECULAR_WEIGHT} g/mol")
    print(f"Database query failed     : {ex}")

CHEMICAL PROPERTIES
Name                      : AMMONIA
Molecular Weight          : 17.03
AEGL-1 (60 min)           : 30 ppm
AEGL-2 (60 min)           : 160 ppm
AEGL-3 (60 min)           : 1100 ppm
ERPG-1                    : 25
ERPG-2                    : 150
ERPG-3                    : 750


## 5) Run Dispersion Simulation

In [16]:
simulation_datetime = datetime.now()
weather = {
    "wind_speed":    WIND_SPEED,
    "wind_dir":      WIND_DIRECTION,
    "temperature_K": TEMPERATURE_C + 273.15,
    "humidity":      HUMIDITY / 100.0,
    "cloud_cover":   CLOUD_COVER / 100.0,
    "source":        "manual",
}
stability_class = get_stability_class(
    wind_speed=weather["wind_speed"],
    datetime_obj=simulation_datetime,
    latitude=SOURCE_LAT,
    longitude=SOURCE_LON,
    cloudiness_index=weather.get("cloud_cover", 0) * 10,
    timezone_offset_hrs=TIMEZONE_OFFSET_HRS,
)
X, Y, _, _ = setup_computational_grid(x_max=X_MAX, y_max=Y_MAX, nx=NX, ny=NY)
release_duration_s = DURATION_MINUTES * 60.0
source_q = RELEASE_RATE
dispersion_mode = "continuous" if SOURCE_TERM_MODE == "continuous" else "instantaneous"
concentration, U_local, resolved_stability, resolved_sources = calculate_gaussian_dispersion(
    weather=weather,
    X=X,
    Y=Y,
    source_lat=SOURCE_LAT,
    source_lon=SOURCE_LON,
    molecular_weight=MOLECULAR_WEIGHT,
    default_release_rate=source_q,
    default_height=TANK_HEIGHT,
    z_ref=3.0,
    z_measurement=RECEPTOR_HEIGHT_M,
    t=release_duration_s,
    t_r=release_duration_s,
    mode=dispersion_mode,
    sources=[{
        "lat": SOURCE_LAT, "lon": SOURCE_LON,
        "name": "Release Source",
        "height": TANK_HEIGHT, "rate": source_q,
        "color": "red",
    }],
    latitude=SOURCE_LAT,
    longitude=SOURCE_LON,
    timezone_offset_hrs=TIMEZONE_OFFSET_HRS,
    roughness=TERRAIN_ROUGHNESS,
    datetime_obj=simulation_datetime,
)
print(f"Simulation time       : {simulation_datetime}")
print(f"Resolved wind speed   : {U_local:.2f} m/s")
print(f"Stability class       : {resolved_stability}")
print(f"Grid shape            : {concentration.shape}")
print(f"Peak concentration    : {float(np.nanmax(concentration)):.2f} ppm")

Simulation time       : 2026-02-23 22:01:15.301415
Resolved wind speed   : 2.50 m/s
Stability class       : F
Grid shape            : (500, 500)
Peak concentration    : 852.65 ppm


## 6) Load Health Thresholds

Fetches thresholds from each selected standard. Standards not found in the database are
skipped with a warning rather than raising an error.

In [17]:
all_thresholds = {}

loaders = {
    "aegl": lambda: get_aegl_thresholds(CHEMICAL_NAME),
    "erpg": lambda: get_erpg_thresholds(CHEMICAL_NAME),
    "pac":  lambda: get_pac_thresholds(CHEMICAL_NAME),
}

for name in SELECTED_SETS:
    name_l = name.lower()
    if name_l == "idlh":
        try:
            idlh_val = get_idlh_threshold(CHEMICAL_NAME)
            if idlh_val is not None:
                all_thresholds["IDLH"] = float(idlh_val)
                print(f"IDLH                  : {float(idlh_val):.1f} ppm")
            else:
                print(f"IDLH                  : not found in database")
        except Exception as ex:
            print(f"IDLH load failed      : {ex}")
    elif name_l in loaders:
        try:
            result = loaders[name_l]()
            if result:
                all_thresholds.update(result)
                for k, v in result.items():
                    print(f"{k:<18}          : {v:.1f} ppm")
            else:
                print(f"{name.upper()} thresholds    : not found in database")
        except Exception as ex:
            print(f"{name.upper()} load failed      : {ex}")
    else:
        print(f"Unknown set           : {name!r} (skipped)")

print()
print(f"Total thresholds loaded : {len(all_thresholds)}")

AEGL-1                      : 30.0 ppm
AEGL-2                      : 160.0 ppm
AEGL-3                      : 1100.0 ppm
ERPG-1                      : 25.0 ppm
ERPG-2                      : 150.0 ppm
ERPG-3                      : 750.0 ppm

Total thresholds loaded : 6


## 7) Extract Zones for All Health Thresholds

Each threshold produces an independent zone polygon from the concentration field.

In [18]:
all_zones = {}
for threshold_name, threshold_value in all_thresholds.items():
    try:
        thr = float(threshold_value)
    except Exception:
        continue
    zones = extract_zones(
        X, Y, concentration,
        {threshold_name: thr},
        SOURCE_LAT, SOURCE_LON,
        wind_dir=weather["wind_dir"],
    )
    all_zones.update(zones)

print(f"Zones extracted         : {len(all_zones)}")
for zone_name, poly in all_zones.items():
    formed = poly is not None and not poly.is_empty
    status = "formed" if formed else "no zone (concentration below threshold)"
    print(f"  {zone_name:<12}: {status}")

Zones extracted         : 6
  AEGL-1      : formed
  AEGL-2      : formed
  AEGL-3      : no zone (concentration below threshold)
  ERPG-1      : formed
  ERPG-2      : formed
  ERPG-3      : formed


## 8) Report Zone Distances

Maximum distance is measured from source to polygon boundary.

In [19]:
print("=" * 72)
print("METEOROLOGICAL CONDITIONS")
print("=" * 72)
print(f"Wind Speed            : {weather['wind_speed']:.2f} m/s")
print(f"Wind Direction        : {weather['wind_dir']:.0f} deg")
print(f"Temperature           : {weather['temperature_K'] - 273.15:.1f} C")
print(f"Humidity              : {weather['humidity'] * 100:.0f} %")
print(f"Cloud Cover           : {weather['cloud_cover'] * 100:.0f} %")
print(f"Stability Class       : {resolved_stability}")
print()
print("=" * 72)
print("HEALTH THRESHOLD ZONE DISTANCES FROM SOURCE")
print("=" * 72)
for zone_name, poly in all_zones.items():
    threshold_val = all_thresholds.get(zone_name, "N/A")
    threshold_str = f"{threshold_val:.0f} ppm" if isinstance(threshold_val, (int, float)) else str(threshold_val)
    dist_km = max_distance_from_source_km(poly, SOURCE_LAT, SOURCE_LON) if poly else None
    if dist_km is not None:
        print(
            f"{zone_name:<12} threshold: {threshold_str:<10} max distance: {dist_km:.3f} km ({dist_km * 1000:.0f} m)"
        )
    else:
        print(f"{zone_name:<12} threshold: {threshold_str:<10} max distance: -- (no zone formed)")

METEOROLOGICAL CONDITIONS
Wind Speed            : 2.50 m/s
Wind Direction        : 90 deg
Temperature           : 25.0 C
Humidity              : 65 %
Cloud Cover           : 30 %
Stability Class       : F

HEALTH THRESHOLD ZONE DISTANCES FROM SOURCE
AEGL-1       threshold: 30 ppm     max distance: 1.769 km (1769 m)
AEGL-2       threshold: 160 ppm    max distance: 0.622 km (622 m)
AEGL-3       threshold: 1100 ppm   max distance: -- (no zone formed)
ERPG-1       threshold: 25 ppm     max distance: 1.991 km (1991 m)
ERPG-2       threshold: 150 ppm    max distance: 0.646 km (646 m)
ERPG-3       threshold: 750 ppm    max distance: 0.204 km (204 m)


## 9) Build Health Impact Map (Inline)

Zone fill colors follow the standard severity convention:

| Standard | Level 1 | Level 2 | Level 3 |
|----------|---------|---------|---------|
| AEGL | yellow | orange | red |
| ERPG | light blue | blue | purple |
| PAC | light green | green | dark green |
| IDLH | dark red | — | — |

In [20]:
THRESHOLD_COLORS = {
    "AEGL-1": "#FFFF66", "AEGL-2": "#FFB84D", "AEGL-3": "#FF6666",
    "ERPG-1": "#ADD8E6", "ERPG-2": "#5599FF", "ERPG-3": "#CC77CC",
    "PAC-1":  "#90EE90", "PAC-2":  "#33BB66", "PAC-3":  "#2D8A5E",
    "IDLH":   "#CC3333",
}

m = folium.Map(location=[SOURCE_LAT, SOURCE_LON], zoom_start=13, tiles="OpenStreetMap")

# Health threshold zone polygons
for zone_name, zone_poly in all_zones.items():
    if zone_poly is None or zone_poly.is_empty:
        continue
    color = THRESHOLD_COLORS.get(zone_name, "#888888")
    folium.GeoJson(
        {"type": "Feature", "geometry": geom_mapping(zone_poly), "properties": {}},
        style_function=lambda _, c=color: {
            "fillColor": c, "color": c, "weight": 2, "fillOpacity": 0.30,
        },
        tooltip=zone_name,
        name=zone_name,
    ).add_to(m)

# Release source marker
folium.Marker([SOURCE_LAT, SOURCE_LON], tooltip="Release Source").add_to(m)

folium.LayerControl().add_to(m)
print(f"Health impact map generated with {len(all_zones)} zones. Displaying inline below.")

Health impact map generated with 6 zones. Displaying inline below.


## 10) Display Health Impact Map Inline

In [21]:
if 'm' in globals() and m is not None:
    display(HTML(m._repr_html_()))
else:
    print("Map object not found. Run Cell 19 first.")

## 11) Inline Results Summary

Quick run summary for tutorial users.

In [22]:
peak_ppm = float(np.nanmax(concentration))
zone_stats = {
    name: max_distance_from_source_km(poly, SOURCE_LAT, SOURCE_LON)
    for name, poly in all_zones.items()
}
zones_formed = sum(1 for d in zone_stats.values() if d is not None)
print("Completed successfully.")
print(f"Chemical              : {CHEMICAL_NAME}")
print(f"Peak concentration    : {peak_ppm:.2f} ppm")
print(f"Threshold sets        : {', '.join(s.upper() for s in SELECTED_SETS)}")
print(f"Total thresholds      : {len(all_thresholds)}")
print(f"Zones formed          : {zones_formed} / {len(all_zones)}")
print("Max zone distances:")
for z, d in zone_stats.items():
    if d is None:
        print(f"  {z}: no zone formed")
    else:
        print(f"  {z}: {d:.3f} km ({d * 1000:.0f} m)")

Completed successfully.
Chemical              : AMMONIA
Peak concentration    : 852.65 ppm
Threshold sets        : AEGL, ERPG
Total thresholds      : 6
Zones formed          : 5 / 6
Max zone distances:
  AEGL-1: 1.769 km (1769 m)
  AEGL-2: 0.622 km (622 m)
  AEGL-3: no zone formed
  ERPG-1: 1.991 km (1991 m)
  ERPG-2: 0.646 km (646 m)
  ERPG-3: 0.204 km (204 m)


## Troubleshooting

- If imports fail, run from repository root **or** install package mode: `pip install pyeldqm`.

- If the map does not render inline, rerun Cell 19 then Cell 21.

- If zones are empty, increase release rate/duration, reduce thresholds, or expand the grid extent.

- If a threshold set reports "not found in database", the chemical may not have that standard
  recorded — remove the set from `SELECTED_SETS` or add the values manually to `all_thresholds`
  before running Cell 17.

- To add PAC or IDLH zones, append `'pac'` or `'idlh'` to `SELECTED_SETS` in Cell 5.